# Standardalgorithmen
Zum Durchlaufen eines Graphen existieren einige Standardalgorithmen, die jeder Data Scientist kennen sollte.

In [ ]:
import math
import networkx as nx
from heapq import heappush, heappop

from tui_dsmt.graph import dach_cities, BFS, DFS, Dijkstra

## Inhaltsverzeichnis
- [Vorbereitung](#Vorbereitung)
- [Traversierung](#Traversierung)
  - [Breitensuche](#Breitensuche)
  - [Tiefensuche](#Tiefensuche)
- [Algorithmus von Dijkstra](#Algorithmus-von-Dijkstra)

## Vorbereitung
Der Beispielgraph enthält ausgewählte Städte aus Deutschland, Österreich und der Schweiz inklusive der ungefähren Distanz zwischen ihnen. (Wie immer ist die Position der Knoten nicht relevant. Es werden nur die Knoten und ihre Verbindungen untereinander betrachtet.)

In [ ]:
pos = nx.spring_layout(dach_cities, seed=9)

nx.draw_networkx_nodes(dach_cities, pos, node_size=700)
nx.draw_networkx_edges(dach_cities, pos)
nx.draw_networkx_labels(dach_cities, pos, font_size=9)
nx.draw_networkx_edge_labels(dach_cities, pos, nx.get_edge_attributes(dach_cities, 'd'))

pass

Im Folgenden wollen wir diesen Graphen durchlaufen. Hannover dient dabei als Ausgangspunkt. Im Laufe der Algorithmen wird dabei auch jeweils ein kürzester Pfad zu allen anderen Knoten berechnet.

## Traversierung
Die Traversierung eines Graphen ist ein grundlegender Algorithmus in der Informatik, der dazu dient, alle Knoten eines Graphen zu besuchen. Diese Operation kann in verschiedene Richtungen erfolgen und dient oft dazu, spezifische Informationen über den Graphen zu sammeln oder um eine bestimmte Suche durchzuführen. Die Traversierung eines Graphen beginnt typischerweise an einem bestimmten Startknoten, von dem aus der Algorithmus beginnt, sich durch den Graphen zu bewegen. Es gibt zwei grundlegende Traversierungsmethoden: die Tiefensuche (DFS) und die Breitensuche (BFS), die jeweils unterschiedliche Ansätze für die Reihenfolge bieten, in der die Knoten des Graphen besucht werden, und jeweils kurz demonstriert werden sollen.

### Breitensuche
Die Breitensuche ist das erste von zwei einfachen Verfahren. Sie sucht - dem Namen entsprechend - zuerst in die Breite, bevor in die Tiefe gegangen wird. Das Verfahren funktioniert wie folgt:
- Allen Knoten wird ein Attribut zugeordnet, dass sie als noch nicht besucht kennzeichnet.
- Es wird eine Liste angelegt, die den Startknoten mit der Distanz $0$ enthält. Der Startknoten wird als besucht markiert.
- Solange die Liste nicht leer ist
  - wird das erste Element entnommen,
  - die aktuelle Entfernung gespeichert und
  - alle durch eine Kante erreichbaren mit der aktuellen Tiefe der Liste hinzugefügt und als besucht markiert, sofern diese nicht bereits zuvor besucht wurden. (Die letzte Bedingung ist relevant in Graphen, die Kreise enthalten.)

Ausgehend vom Startknoten werden dann zuerst alle Knoten besucht, die eine Kante entfernt liegen, dann alle Knoten, die zwei Kanten entfernt liegen, usw...

In [ ]:
# "besucht" und "Entfernundach_cities" setzen
for node in dach_cities:
    dach_cities.nodes[node]['visited'] = False
    dach_cities.nodes[node]['distance'] = 0

# Besuchsliste anlegen und Startknoten als besucht markieren
visit = [('Hannover', 0)]
dach_cities.nodes['Hannover']['visited'] = True

# solange die Liste nicht leer ist
while len(visit) > 0:
    # erstes Element entfernen
    current_node, current_distance = visit.pop(0)
    print(current_distance, current_node)

    # als besucht markieren und Entfernung setzen
    dach_cities.nodes[current_node]['distance'] = current_distance

    # alle nicht besuchten Nachbarn der Liste hinzufügen
    for neighbor in dach_cities.neighbors(current_node):
        if not dach_cities.nodes[neighbor]['visited']:
            dach_cities.nodes[neighbor]['visited'] = True
            visit.append((neighbor, current_distance + 1))

Da die Knoten in der Reihenfolge der Entfernung besucht werden, findet die Breitensuche kürzeste Wege zu jedem erreichbaren Knoten. Knoten, die nach dem Ende des Algorithmus nicht besucht wurden, können nicht vom Startknoten aus erreicht werden und sind folglich auch nicht mit diesem durch einen Pfad verbunden.

In der nachfolgenden Zelle haben Sie die Möglichkeit, dem Algorithmus Schritt für Schritt bei der Arbeit zuzusehen.

In [ ]:
BFS(dach_cities, start_node='Hannover')

### Tiefensuche
Die Tiefensuche hingegen sucht dem Namen entsprechend zuerst in der Tiefe. Dafür wird ausgehend vom Startknoten ein Weg solange verfolgt, bis keine nicht mehr besuchten Nachbarn gefunden werden können. Danach wird nur so weit zurückgegangen, bis durch eine andere Kante wieder in die Tiefe gesucht werden kann.

Der Algorithmus wird in der Regel rekursiv implementiert und verläuft nun wie folgt:
- Alle Knoten werden als nicht besucht markiert.
- Vom Startknoten ausgehend wird eine Funktion aufgerufen, die
  - den Knoten als besucht markiert.
  - sich selbst rekursiv für alle Nachbarknoten aufruft, die noch nicht besucht wurden.

In Graphen mit Zyklen findet die einfache Tiefensuche nicht zwangsläufig einen kürzesten Pfad.

In [ ]:
# "besucht" und "Entfernung" setzen
for node in dach_cities:
    dach_cities.nodes[node]['visited'] = False

# Besuchsfunktion
def visit(node, distance=0):
    print('  ' * distance, node, sep='')

    # Knoten als besucht markieren
    dach_cities.nodes[node]['visited'] = True

    # rekursiver Aufruf für noch nicht besuchte Nachbarn
    for neighbor in dach_cities.neighbors(node):
        if not dach_cities.nodes[neighbor]['visited']:
            visit(neighbor, distance+1)

# initialer Aufruf mit Startknoten
visit('Hannover')

Die Einrückung repräsentiert die Tiefe des rekursiven Aufrufs. Zu erkennen ist, dass zuerst maximal in die Tiefe gegangen wird, bevor der Algorithmus zurückkehrt und andere Nachbarn besucht. Auch bei diesem Algorithmus sind Knoten nicht mit dem Startknoten verbunden, wenn sie nach dem Ende noch nicht besucht wurden.

In der nachfolgenden Zelle haben Sie die Möglichkeit, dem Algorithmus Schritt für Schritt bei der Arbeit zuzusehen.

In [ ]:
DFS(dach_cities, start_node='Hannover')

## Algorithmus von Dijkstra
Der Dijkstra-Algorithmus ist ein effizienter Algorithmus (schneller als die Breitensuche) zur Bestimmung des kürzesten Weges in einem gewichteten Graphen von einem Startknoten zu allen anderen Knoten. Der Algorithmus funktioniert wie folgt:

1. Zuerst erhalten alle Knoten zwei zusätzliche Attribute, welche die minimal mögliche Entfernung zum Startknoten und die Erreichbarkeit repräsentieren sollen. Initialisiert wird dieses Attribut mit $0$ für den Startknoten und $\infty$ für alle anderen Knoten. Eine Prioritätswarteschlange wird verwendet, um die Knoten mit ihren aktuellen Abstandswerten zu verfolgen.
2. Der Algorithmus wählt den Knoten mit dem geringsten Abstandswert aus der Prioritätswarteschlange aus. Dieser Knoten wird als besucht markiert und aus der Warteschlange entfernt.
3. Für jeden benachbarten Knoten des aktuellen Knotens überprüft der Algorithmus, ob der Pfad über den aktuellen Knoten zu diesem Nachbarn kürzer ist als der bisher bekannte Weg. Wenn ja, wird der Abstandswert des Nachbarn aktualisiert und der Knoten mit seinem neuen Abstandswert in die Prioritätswarteschlange eingefügt.
4. Nach Abschluss des Algorithmus enthält jeder Knoten den Abstandswert zum Startknoten und damit den kürzesten Weg von diesem Knoten aus. Der Algorithmus liefert entweder die kürzesten Wege zu allen anderen Knoten oder den kürzesten Weg zu einem spezifischen Ziel.

In [ ]:
# Alle Knoten werden vorbereitet, indem die Attribute
# visited und distance hinzugefügt werden.
for node in dach_cities:
    dach_cities.nodes[node]['visited'] = False
    dach_cities.nodes[node]['distance'] = 0 if node == 'Hannover' else math.inf

# Die Prioritätenliste wird initialisiert, indem
# der Startknoten mit Distanz 0 hinzugefügt wird.
queue = [(0, 'Hannover')]

# Der Algorithmus läuft, solange die Prioritäten-
# liste nicht leer ist.
while len(queue) > 0:
    # Der nächste Knoten und seine Distanz werden
    # aus der Prioritätsliste entnommen.
    node_distance_to_start, node = heappop(queue)

    # Falls der Knoten bereits besucht wurde,
    # wird er übersprungen. (Das kann z.B.
    # der Fall sein, falls zwischenzeitlich
    # ein kürzerer Weg zu diesem Knoten
    # gefunden wurde.) Danach wird der Knoten
    # als besucht markiert.
    if dach_cities.nodes[node]['visited']:
        continue

    dach_cities.nodes[node]['visited'] = True

    # Jeder Nachbar des aktuellen Knotens
    # soll geprüft werden.
    for neighbor in dach_cities.neighbors(node):
        # Die Distanz zwischen Start- und
        # dem jeweiligen Nachbarknoten über
        # den aktuellen Knoten wird berechnet.
        node_distance_to_neighbor = dach_cities.get_edge_data(node, neighbor)['d']
        neighbor_distance_to_start = node_distance_to_start + node_distance_to_neighbor

        # Nur falls die neue Distanz zwischen
        # Start- und Nachbarknoten geringer
        # als die bisher bekannte ist, wird
        # die gespeicherte Distanz aktualisiert
        # und der Nachbarknoten mit seiner
        # neuen Distanz in die Prioritäten-
        # liste eingefügt. (Jeder verbundene
        # Knoten wird durch die
        # Initialisisierung mit inf mindestens
        # einmal aktualisiert.)
        if neighbor_distance_to_start < dach_cities.nodes[neighbor]['distance']:
            dach_cities.nodes[neighbor]['distance'] = neighbor_distance_to_start
            heappush(queue, (neighbor_distance_to_start, neighbor))

# Sobald der Algorithmus endet, enthält das
# Distanzattribute jeweils die minimale
# Distanz eines Knotens zum Startknoten.
nx.get_node_attributes(dach_cities, 'distance')

Auch diesem Algorithmus können Sie schrittweise bei der Arbeit verfolgen:

In [ ]:
Dijkstra(dach_cities, 'Hannover')